In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Generating the dataset {-}

In [ ]:
def generate(k):
    
    # Initializing the constants. m here represents the number of features and not the number of datapoints.
    m, mu, sigma_sq = 20, 0, 0.1
    X, Y = np.zeros([k, m+1]), np.zeros([k,])
    
    for j in range(k):
        x_val, y_val = [], []
        
        # The first time is the bias
        x_val.append(1)
        
        def generate_x():
            for i in range(m):
                if i < 11 or i > 15:
                    x_val.append(np.random.normal(1, sigma_sq))
                elif i == 11:
                    x_val.append(x_val[0] + x_val[1] + np.random.normal(mu, sigma_sq))
                elif i == 12:
                    x_val.append(x_val[2] + x_val[3] + np.random.normal(mu, sigma_sq))
                elif i == 13:
                    x_val.append(x_val[4] + x_val[5] + np.random.normal(mu, sigma_sq))
                elif i == 14:
                    x_val.append((0.1 * x_val[6]) + np.random.normal(mu, sigma_sq))
                elif i == 15:
                    x_val.append((2 * x_val[2]) - 10 + np.random.normal(mu, sigma_sq))
            X[j] = np.array(x_val)


        def generate_y(x):
            for i in range(1, 11):
                a = ((0.6 ** i) * x[i]) + np.random.normal(mu, sigma_sq)
            Y[j] = 10 + a

            
        generate_x()
        generate_y(x_val)
    plt.scatter([p+1 for p in range(k)], Y)
    plt.show()
    return X, Y

In [ ]:
# The parameter passed here is the 'm', i.e., number of datapoints.
lr_x, lr_y = generate(1000)

# Writing a naive Linear Regression class and generating weights {-}

In [ ]:
class Linear_Regression:
    def __init__(self):
        pass
    
    def naive_linear_regression(self, x, y):
        # Getting the number of datapoints and number of features.
        m, k = x.shape
        
        # Initializing the weights to be zeros.
        self.weights = np.zeros((1, k))
        
        x_transpose = np.transpose(x)
        # Applying the linear regression model to generate weights.
        self.weights = np.dot(np.dot(np.linalg.inv(np.dot(x_transpose, x)), x_transpose), y)
        return self.weights
    
    # This function is used to predict using the existing weights.
    def predict(self, x, weight):
        return np.dot(x, weight)

In [ ]:
# Training a model and getting the weights.
lr = Linear_Regression()
lr_weights = lr.naive_linear_regression(lr_x, lr_y)

## Plotting the weights {-}

In [ ]:
print(lr_weights)
plt.scatter([i for i in range(21)], lr_weights)
plt.show()

## Comparison with "True weights and biases" {-}

From our true model, we can see that the true bias value is 10. Also, we can see that the true weights would be $(0.6)^i$. 
As our bias value is the first term in weights, we can compare it with the true bias.

In [ ]:
true_bias = 10
model_bias = lr_weights[0]
print("The difference between true bias and the bias from the model is {}".format(true_bias-model_bias))

This says that our bias is a bit larger and the amount by which it is larger is 0.1868.

## Most and least significant features {-}

In [ ]:
print("Most significant feature is {}".format(np.argmax(lr_weights[1:])+1))
print("Least significant feature is {}".format(np.argmin(lr_weights[1:])+1))

## Finding the training error {-}

In [ ]:
prediction = lr.predict(x, lr_weights)
# Using L2 norm to find the error in the predictions.
print("Training error is {}%".format(np.linalg.norm(prediction - y)/1000))

## Testing our model on new data {-}

In [ ]:
test_x, test_y = generate(1000)
predict1 = lr.predict(test_x, weights)
print("Testing error is {}%".format(np.linalg.norm(predict1 - test_y)/1000))

## Checking if any weights are pruned {-}

In [ ]:
count = 0
for check_weight in lr_weights:
    if check_weight == 0:
        count += 1
print(count)

As we can see, there is no feature whose weight is 0. As a result, we can say that our algorithm did not prune any weight.

# Ridge Regerssion {-}

In [ ]:
class Ridge_Regression:
    def __init__(self, lam):
        # Passing the lambda value
        self.lam = lam
    
    def ridge_regression(self, x, y):
        # Getting the number of datapoints and number of features.
        m, k = x.shape
        identity = np.identity(k)
        
        # Initializing the weights to be zeros.
        self.weights = np.zeros((1, k))
        
        x_transpose = np.transpose(x)
        # Applying the linear regression model to generate weights.
        self.weights = np.dot(np.dot(np.linalg.inv(np.dot(x_transpose, x) + (self.lam * identity)), x_transpose), y)
        return self.weights
    
    # This function is used to predict using the existing weights.
    def predict_ridge(self, x, weight):
        return np.dot(x, weight)

In [ ]:
# Training a model and getting the weights.
rr = Ridge_Regression(0.0035)
rr_weights = rr.ridge_regression(lr_x, lr_y)

In [ ]:
ridge_prediction = rr.predict_ridge(lr_x, rr_weights)
# Using L2 norm to find the error in the predictions.
print("Training error is {}%".format(np.linalg.norm(ridge_prediction - lr_y)/1000))

## Testing the Ridge Regression model on a large dataset {-}

In [ ]:
test_rr_x, test_rr_y = generate(10000)
ridge_testing = rr.predict_ridge(test_rr_x, rr_weights)
print("Testing error for ridge regression is {}%".format(np.linalg.norm(ridge_testing - test_rr_y)/10000))

## Plotting against lambda for m = 1000 {-}

In [ ]:
lambda_value = -1
error_list, m_list = [], []
while lambda_value <= 5:
    lambda_rr = Ridge_Regression(lambda_value)
    lambda_rr_weights = lambda_rr.ridge_regression(lr_x, lr_y)
    ridge_pred = rr.predict_ridge(lr_x, lambda_rr_weights)
    error = np.linalg.norm(ridge_pred - lr_y)/1000
    m_list.append(lambda_value)
    error_list.append(error)
    lambda_value += 0.1

plt.plot(m_list, error_list)
plt.show()